<a href="https://colab.research.google.com/github/sanjit1995/MovieGenreClassifier/blob/master/MovieGenreClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/MovieGenreClassification/combinedFromTmdbForNLP.csv')

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df.isnull().sum()

index          0
genres         0
overview       0
genre_ids      0
genre_names    0
dtype: int64

In [ ]:
df = df.dropna(how='any')
df = df.reset_index()
df = df.drop(['index'], axis=1)

In [ ]:
genres_df = df['genres']

In [ ]:
ind = 0
for temp_genres in genres_df:
  genre_ids = []
  genre_names = []
  print(temp_genres)
  genres = eval(temp_genres)
  for i in range (0, len(genres)):
    genre_ids.append(str(genres[i]['id']))
    genre_names.append(genres[i]['name'])
  df.at[ind,'genre_ids'] = ','.join(map(str, genre_ids))
  df.at[ind,'genre_names'] = ",".join(genre_names)
  ind += 1

In [ ]:
df

In [ ]:
df.columns

Index(['genres', 'overview', 'genre_ids', 'genre_names'], dtype='object')

In [ ]:
train_data = df.reset_index()[['overview','genre_names']]
train_data.columns = ['summary', 'genres']

In [ ]:
train_data

,summary,genres
0,"When Lou, who has become the ""father of the In...",Comedy
1,Mia Thermopolis is now a college graduate and ...,"Comedy,Drama,Family,Romance"
2,"Under the direction of a ruthless instructor, ...",Drama
3,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,"Thriller,Drama"
4,Marine Boy is the story of a former national s...,"Action,Thriller"
...,...,...
7350,A young man is plunged into a life of subterfu...,Thriller
7351,"Three friends, whose lives have been drifting ...","Action,Adventure,Comedy,Thriller"
7352,"Frank Galvin is a down-on-his luck lawyer, red...",Drama
7353,"For 19-year-old Jay, fall should be about scho...","Horror,Thriller"


In [ ]:
train_data.isnull().sum()

In [ ]:
train_data['genres'] = train_data['genres'].str.replace(r'|', ',')
train_data['genres'] = train_data['genres'].str.replace(r'\\N', '')

In [ ]:
train_data

In [ ]:
x_train = train_data[['summary']]
y_train = train_data['genres'].tolist()

In [ ]:
x_train.shape

(7355, 1)

In [ ]:
y_train[0]

'Comedy'

get count of each genre

In [ ]:
list_temp = []
genres_count = {}
for elem in y_train:
  temp_list = []
  temp_list = str(elem).split(',')
  for genre in temp_list:
    genres_count.setdefault(genre,0)
    genres_count[genre] += 1
genres_count

In [ ]:
genres_count

In [ ]:
list_temp

In [ ]:
y_list = []
for elem in y_train:
  temp_list = []
  temp_list = str(elem).split(',')
  y_list.append(temp_list)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
one_hot = MultiLabelBinarizer()
y_list = one_hot.fit_transform(y_list)

In [ ]:
one_hot.classes_

array(['Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History',
       'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
       'TV Movie', 'Thriller', 'War', 'Western'], dtype=object)

In [ ]:
len(y_list)

7355

In [ ]:
y_list.value_counts()

In [ ]:
import re
x_train['filtered_summary'] = x_train['summary'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'pic.\S+', '', regex=True).replace(r'photos.app\S+', '', regex=True).replace(r'twitter.com\S+', '', regex=True)
x_train['filtered_summary'] = x_train['filtered_summary'].map(lambda x: re.sub(r'[^A-Za-z\s\#\-\']+', '', x))
x_train['filtered_summary'] = x_train['filtered_summary'].str.replace(r'\(.*\)','')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

x_train['final_summary'] = x_train['filtered_summary'].apply(lambda x: remove_stopwords(x))

In [ ]:
x_train

In [ ]:
y_list

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    min_df=1,  # min count for relevant vocabulary
    max_features=15000,  # maximum number of features
    # by their corresponding  ASCII char
    lowercase=True,
    analyzer='word',  # features made of words
    ngram_range=(1, 3),  # features made of a single tokens
    use_idf=True,  # enable inverse-document-frequency reweighting
    smooth_idf=True,  # prevents zero division for unseen words
    sublinear_tf=False)

tfidf_df = tfidf_vectorizer.fit_transform(x_train['final_summary'])

In [ ]:
tfidf_df.shape

(7355, 15000)

In [ ]:
y_list.shape

(7355, 20)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
classifier.fit(tfidf_df, y_list)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier()
classifier.fit(tfidf_df, y_list)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
classifier = OneVsRestClassifier(LogisticRegression())
classifier.fit(tfidf_df, y_list)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [ ]:
test_string = {"Chhatrapati Shivaji's fight war right-hand man, braveheart Subhedar Tanhaji Malusare, pits his sharp acumen against the brawn of ruthless and hedonistic Mughal chieftain Udaybhan Singh Rathore to recapture Kondhana for the Maratha empire."}
test = tfidf_vectorizer.transform(test_string)
one_hot.inverse_transform(classifier.predict(test))

[('Action',)]

In [ ]:
test

<1x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 37 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_vectorizer.get_feature_names()

'abandoned'

In [ ]:
for i in tfidf_vectorizer.get_feature_names():
  print(i)
  if i == 'the':
    break

In [ ]:
tfidf_df[42]

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 51 stored elements in Compressed Sparse Row format>

In [ ]:
classifier.predict(tfidf_df[41])

array([[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
classifier.predict(test)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
list(one_hot.inverse_transform(classifier.predict(tfidf_df[41])))

[('Action', 'Drama')]

In [ ]:
one_hot.inverse_transform(classifier.predict(test))

[()]

In [ ]:
df.iloc[41][['genres','title']]